<a href="https://colab.research.google.com/github/MariamChowdhury/FoR_Lab_2025-2026/blob/main/lab1_firstMIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to the _mip_ Python module

The module we'll become most familiar with in this course is the `mip` module, which allows for creating, manipulating, and solving optimization models with linear constraints and integer, binary, or continuous variables. Check the [homepage](https://www.python-mip.com) for full access to the documentation and updates.

Suppose you want to model the following problem:
$$
\begin{array}{ll}
  \max & x_1 + x_2\\
  \textrm{s.t.} & 2 x_1 + x_2 \le 10\\
  & x_1, x_2 \ge 0
\end{array}
$$

For starters, we import the module `mip` in Python.

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.1/484.1 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 99.4 MB/s eta 0:00:00
  Created wheel for cffi: filename=cffi-1.15.0-cp312-cp312-linux_x86_64.whl size=400939 sha256=e4abe37dd0d9959ac76f69a717739d179b1204aa6dc05af15496a1fea586d760
  Stored in directory: /root/.cache/pip/wheels/b9/d6/15/0950847bf7d74ea5f0380b8b23a1d81b45bdf48488b4b8237a
Successfully built cffi
  Attempting uninstall: cffi
    Found existing installation: cffi 2.0.0
    Uninstalling cffi-2.0.0:
      Successfully uninstalled cffi-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rpy2 3.5.17 requires cffi>=1.15.1, but you have cffi 1.15.0 which is incompatible.
pygit2 1.19.0 requires cffi>=2.0, but you have cffi 1.15.0 which is incompatible.


In [ ]:
import mip

Next, we create an optimization model `m`. We do so by calling the `mip.Model` *constructor* method. We also create two variables `x1` and `x2` using the `add_var()` method from the optimization model.

In [ ]:
m = mip.Model()

x1 = m.add_var()
x2 = m.add_var()

#print(m)

We now add the single constraint and the objective.

To add the constraint, we use the method `add_constr` from the optimization model.
To add the objective function, we set the `objective` attribute of `m`. We use the method `mip.maximize`, to indicate that this is a function to be maximized.

For now, since both the constraint and the objective are very simple, we fully write them as algebraic expressions of `x1` and `x2`.

In [ ]:
m.add_constr(2*x1 + x2 <= 10)

m.objective = mip.maximize(x1 + x2)

Finally, we call the method `optimize` to solve the problem and print the value of the optimal solution. For a variable `v` of the module `mip`, its value in the optimal solution is retrieved as the attribute `.x`, for example `v.x`.

In [ ]:
m.optimize()

print('solution:', x1.x, ',', x2.x)

solution: 0.0 , 10.0


Here's the complete program.

In [ ]:
#template jeita sob mip er jonne use kora hobe

import mip #import the package

m = mip.Model() #create the empty model

x1 = m.add_var() #add variables
x2 = m.add_var() #add variables

m.objective = mip.maximize(x1 + x2) #add objective
m.add_constr(2*x1 + x2 <= 10) #add constraints

m.optimize() #and optimize

print('solution:', x1.x, ',', x2.x)

solution: 0.0 , 10.0


# A slightly more advanced example

Let us now consider a slightly more complicated example: formulating and solving a knapsack problem.

$$
\begin{array}{lll}
\max & 3 x_1 + 4 x_2 + 7 x_3 + 5 x_4\\
\textrm{s.t.} & 4 x_1 + 5 x_2 + 6 x_3 + 4 x_4 \le 13\\
              & x_1, x_2, x_3, x_4 \in \{0,1\}
\end{array}
$$

For starters, we import the module and define the data used in this model.

In [ ]:
# First: import the Python-MIP module
import mip

# Second, define data: two vectors for value and weight and a scalar
# for the right-hand side of the only constraint
value = [3, 4, 7, 5]
weight = [4, 5, 6, 4]
max_weight = 13

Next, we create an optimization model with the `mip.Model` *constructor* method.

We also add four variables using a list, and call that list `x`. Note that we are using a so-called _list comprehension_ to create variables, i.e., we put a `for` construct _inside_ the list in order to create as many list elements as there are numbers in `range(4)`. As you may have gathered from previous cells, `range(4)` is the set of numbers `0, 1, 2, 3`.

In [ ]:
# Create model
m = mip.Model()

x = [m.add_var(var_type=mip.BINARY) for i in range(4)]

We now add the single constraint and the objective. In order to create the sum $\sum_i w_i x_i$, the method `mip.xsum` houls be used. As an argument, one again uses a `for` construct inside the `xsum` argument. The expression

```python
weight[i] * x[i] for i in range(4)
```

generates all products $w_ix_i$ for all $i\in \{0,1,2,3\}$ (I know it might be tricky for many to get used to the idea that indices begin at zero in Python, but this will come in handy in the future). This expression is then wrapped inside a `mip.xsum`, which is constrained to be lesser than or equal to `max_weight`. This is the constraint. It is added to the model with the `+=` operator, which is common in Python and other languages such as C/C++ or Java; `a += b` means "add `b` to `a` and store the result in `a`".

The objective function is a similar `mip.xsum` construction, this time with `value[i]` instead of `weight[i]` for coefficients. It is assigned as the model's objective function with the method `mip.maximize`, to indicate that this is obviously a function to be maximized.

In [ ]:
m.add_constr(mip.xsum(weight[i] * x[i] for i in range(4)) <= max_weight)

m.objective = mip.maximize(mip.xsum(value[i] * x[i] for i in range(4)))

Finally, we call the method `optimize` to solve the problem and print the value of the optimal solution. For a variable `v` of the module `mip`, its value in the optimal solution is retrieved as the attribute `.x`, for example `v.x`.

In [ ]:
m.optimize()

print([x[i].x for i in range(4)])

[1.0, 1.0, 0.0, 1.0]


And here is the whole model in all its glory.

In [ ]:
import mip

# Second, define data: two vectors for value and weight and a scalar
# for the right-hand side of the only constraint
value = [3, 4, 7, 5]
weight = [4, 5, 6, 4]
max_weight = 13

# Create model
m = mip.Model(sense=mip.MAXIMIZE)

#when variables are continous we can keep it empty but eikhane variables binary tai type binary lekha lagbe
#var_type=mip.CONTINOUS => eita default setting, so eita likha and na likha same
#var_type=mip.INTEGER => eita arekta type
#used 1 line with vector? ei line er meaning ask gpt

x = [m.add_var(var_type=mip.BINARY) for i in range(4)]

#multiplied using vector and used a loop?
m.add_constr(mip.xsum(weight[i] * x[i] for i in range(4)) <= max_weight)
m.objective = mip.maximize(mip.xsum(value[i] * x[i] for i in range(4)))

m.optimize()

print([x[i].x for i in range(4)])

[1.0, 1.0, 0.0, 1.0]


Obviously, we could have just formulated the non-parametric version of the problem. Here it is:

In [ ]:
# First: import the Python-MIP module
import mip

# Create model
m = mip.Model(sense=mip.MAXIMIZE)

x1 = m.add_var(var_type=mip.BINARY)
x2 = m.add_var(var_type=mip.BINARY)
x3 = m.add_var(var_type=mip.BINARY)
x4 = m.add_var(var_type=mip.BINARY)

m.add_constr(4*x1 + 5*x2 + 6*x3 + 4*x4 <= max_weight)
m.objective = mip.maximize(3*x1 + 4*x2 + 7*x3 + 5*x4)

m.optimize()

print("Solution:", x1.x, "", x2.x, "", x3.x, "", x4.x)

# If you feel a bit more adventurous, try to understand what the following means:
# print ("Solution: {0}, {1}, {2:.8f}, {3}".format(x1.x, x2.x, x3.x, x4.x))

## Miscellanea and troubleshooting

After this first MIP model it's time to say something more about Python.

### Re-running code on Jupyter notebooks
Code on Jupyter notebooks is fed into Python one cell at a time. If the notebook is written correctly, you should be able to click into the first cell, then just do a `shift`+`enter` through the last cell without any error.

You are also able to re-run any cell multiple times, in any sequence you want. However, be aware that Python sees a sequence of cells it is given, and does not know whether an instruction should be undone or not. Therefore, once a cell is run, its results are _persistent_, at least until we reset them. One big red button is the __Restart__ command under the _Kernel_ tab in the menu: it clears all memory of whatever was done in the cell so far (though obviously not file operations). Later in this notebook we show an example of the trouble persistence can cause.

### Indentation
Indentation is crucial: in a `for` loop, an `if` block, or a function definition, the inner part __MUST__ be indented consistently. Python will throw an error in the following cases:

```python
for i in [1,2,3]:
print(i)
```
Here the `print` instruction should be indented by at least one space.
```python
if i==4:
    print('i is 4')
  print('deal with it')
```
Here indentation is inconsistent.
```python
def myfunction(a):
return a**4 + 5*a**3
```
Same as the first incorrect example. The correct way to write these examples is as follows:
```python
for i in [1,2,3]:
    print(i)

if i==4:
    print('i is 4')
    print('deal with it')

def myfunction(a):
    return a**4 + 5*a**3
```
The suggested indentation is 4 characters.

### Assignment vs. equality
The sign `=` is for _assignment_, while `==` is for checking equality of two expressions. You can write `if a == 4` but not `if a = 4`. Also, writing the statement `a = 4` is correct, and so is `a == 4`; however, the latter has no effect (apart from returning `True` or `False` on the Python command line).

### Semicolons, be gone!
You may have noticed that Python doesn't require semicolons (`;`) at the end of each instruction, as other languages like C, C++, Java, AMPL do. This makes for more readable and prettier code, but indentation is enforced with this in mind.

### Writing a statement on multiple lines
Related to the last point: conditions can be split on multiple lines as long as a `\` is added at the end of all but the last one, for example:
```python
if i==3 or \
   i==4:
    print('i is not 5')
```
But the `\` is not necessary if there is an unclosed parenthesis, for instance:
```python
if (i==3 or i==5 or
    i==7):
    print('i is prime')
```
### If you're feeling a bit masochistic...
A good way to check if your Python program was written according to the standard is to run the `flake8` module on it. Just run `flake8 myprogram.py` and check all the errors it throws (there are usually a ton).

## Persistence and debugging in Jupyer notebooks

Suppose you want to model the following problem:
$$
\begin{array}{ll}
  \max & x_1 + x_2\\
  \textrm{s.t.} & 2 x_1 + x_2 \le 10\\
  & x_1, x_2 \ge 0
\end{array}
$$
Let's write the model using `mip`:

In [ ]:
import mip

m = mip.Model()

x1 = m.add_var(name='x1')
x2 = m.add_var(name='x2')

m.objective = mip.maximize(x1 + x2)
m.add_constr(2*x1 + x2 <= 10)

m.optimize()

print('solution:', x1.x, ',', x2.x)

solution: 0.0 , 10.0


Suppose now we want to relax the constraint, for instance change the right-hand side to 20:

In [ ]:
m.add_constr(2*x1 + x2 <= 20)

m.optimize()

print('solution:', x1.x, ',', x2.x)

solution: 0.0 , 10.0


The solution is the same even though we relaxed the problem. Why? Well, the problem has two constraints: the one we added in the first cell (which is the more restrictive one) and the last constraint. If we want to relax a problem or change it otherwise, we should modify the cell it is contained in.